## Install transformers ##

In [ ]:
!pip install transformers

In [29]:
import pandas as pd
import numpy as np

## Load file ##

In [ ]:
df = pd.read_csv("/content/SMSSpamCollection",sep="\t", names= ["label", "message"])
df

## Feature Extraction ##

In [ ]:
#X = list(df['message'])   Independent variables
X = list(df['message'])
X

In [ ]:
y = list(df['label'])
y

### Mapping ##

In [ ]:
y = pd.get_dummies(y,drop_first=True)['spam']
y

## Train-test split ##

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

## Import Bert tokenizer ##

In [35]:
from transformers import BertTokenizer

In [36]:
model_name="bert-large-uncased"
# Load the BERT large tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

## Get encodings ##
train the features to the BERT model

In [37]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [38]:
#train_encodings
#test_encodings

## Convert encodings into Dataset objects ##

In [39]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [40]:
#train_dataset

## import BERT model ##

In [41]:
from transformers import  BertForMaskedLM, TFTrainer, TFTrainingArguments

In [44]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,
)

# Training the model with TFTrainer #

In [45]:
with training_args.strategy.scope():
   model = BertForMaskedLM.from_pretrained('bert-large-uncased')

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: ignored

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
output = trainer.predict(test_dataset)[1]
output